In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from random import random

In [3]:
#Load ratings csv file

ratings_df = pd.read_csv("ratings_small.csv")#need to try this on full dataset
ratings_df.shape
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


Load Movies metadata csv file

In [4]:
movies_df = pd.read_csv("input/movies_metadata.csv")
movies_df.head(2)

C:\Users\Kathleen\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


Merge the two dataframe to keep only userId, movieId, rating and title data

In [7]:
movies_df = movies_df[movies_df['id'].astype(str).str.isdigit()] # clean (Some movies have not integers as id)
movies_df.id = movies_df.id.astype(np.int64)

ratings_df = pd.merge(ratings_df,movies_df[['title','id']],left_on='movieId',right_on='id')
ratings_df.head(2)

,userId,movieId,rating,timestamp,title_x,id_x,title_y,id_y,title,id
0,1,1371,2.5,1260759135,Rocky III,1371,Rocky III,1371,Rocky III,1371
1,4,1371,4.0,949810302,Rocky III,1371,Rocky III,1371,Rocky III,1371


In [8]:
ratings_df.drop(['timestamp','id'],axis=1,inplace=True)
ratings_df.sample(2)

,userId,movieId,rating,title_x,id_x,title_y,id_y,title
41718,312,1392,4.0,Born into Brothels,1392,Born into Brothels,1392,Born into Brothels
30520,643,4701,2.0,A Tale of Two Cities,4701,A Tale of Two Cities,4701,A Tale of Two Cities


Let's see the number of ratings for each movie

In [9]:
# ??? what is ratings_top

Let's see some statistics for the totalRatings

In [63]:
ratings_top.shape

ratings_top.head()

,userId,movieId,rating,title,totalRatings
0,1,1371,2.5,Rocky III,47
1,4,1371,4.0,Rocky III,47
2,7,1371,3.0,Rocky III,47
3,19,1371,4.0,Rocky III,47
4,21,1371,3.0,Rocky III,47


Make data consistent by ensuring there are unique entries for [title,userId] pairs

In [65]:
if not ratings_top[ratings_top.duplicated(['userId','title'])].empty:
    ratings_top = ratings_top.drop_duplicates(['userId','title'])

ratings_top.shape

(34412, 5)

Reshape the data using pivot function

In [66]:
df_for_knn = ratings_top.pivot(index='title',columns='userId',values='rating').fillna(0)

In [67]:
df_for_knn.tail(15)

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
Will Penny,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
Willy Wonka & the Chocolate Factory,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
Windows on Monday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wings of Desire,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
Wish You Were Here,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Within the Woods,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X-Men Origins: Wolverine,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
Y Tu Mamá También,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yankee Doodle Dandy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
titles = [df_for_knn.index[i] for i in range(len(df_for_knn))]

def neighbors(df, user = None, user_col = None):

    if user is None:
        distances = [np.linalg.norm(df[user_col] - df[u]) for u in df.columns]
        distances_sorted = np.sort(distances)
        nearest = [distances.index(distances_sorted[i]) for i in range(len(df))]
        
    else:
        distances = [np.linalg.norm(user - df[u]) for u in df.columns]
        distances_sorted = np.sort(distances)
        nearest = [distances.index(distances_sorted[i]) for i in range(len(df))]
        
    return nearest

def find_neighbors(df = df_for_knn, user = None, user_col = 0):
    if user is None:
        return neighbors(df, user_col = user_col)
    else:
        return neighbors(df, user = user)
    
def find_recs(df = df_for_knn, user = None, user_col = 0, k = 1):
    neighbor = []
    
    if user is None:
        neighbor = find_neighbors(df, user_col = user_col)
    else:
        neighbor = find_neighbors(df, user = user)
    
    scores = [(i,score) for i,score in enumerate(neighbor) if user[i] == 0]
    scores.sort(key = lambda x:x[1])
    
    return [titles[scores[i][0]] for i in range(k)]
    
random_user = [int(random()*4+1) if random() >= 0.9 else 0 for _ in range(len(df_for_knn))]

temp = find_recs(user = random_user, k = 10)

print(temp)

['300', "Jacob's Ladder", 'The Motorcycle Diaries', 'Jay and Silent Bob Strike Back', 'Sissi', 'The Dark', 'The Dawn Patrol', 'My Name Is Bruce', 'Forbidden Planet', 'Forces of Nature']
